<a href="https://colab.research.google.com/github/kay-kay-t/CodingDojo/blob/main/04_Logistic%2C_Random_forest%2C_or_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary:

1. Type
2. Alcohol
3. Malic
4. Ash
5. Alcalinity
6. Magnesium
7. Phenols
8. Flavanoids
9. Nonflavanoids
10. Proanthocyanins
11. Color
12. Hue
13. Dilution
14. Proline


In [70]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
filename = '/content/drive/MyDrive/Data/wine.data'

In [71]:
# Load data
wine = pd.read_csv(filename, header=None)
wine.rename(columns={0 : 'Type', 1 : 'Alcohol', 2 : 'Malik', 3 : 'Ash', 4 : 'Alcalinity', 5 : 'Magnesium', 6 : 'Phenols', 7 : 'Flavanoids', 8 : 'Nonflavanoid',
                      9 : 'Proanthocyanins', 10 : 'Color', 11 : 'Hue', 12 : 'Dilution', 13 : 'Proline'}, inplace=True)
wine

,Type,Alcohol,Malik,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoid,Proanthocyanins,Color,Hue,Dilution,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [72]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Type             178 non-null    int64  
 1   Alcohol          178 non-null    float64
 2   Malik            178 non-null    float64
 3   Ash              178 non-null    float64
 4   Alcalinity       178 non-null    float64
 5   Magnesium        178 non-null    int64  
 6   Phenols          178 non-null    float64
 7   Flavanoids       178 non-null    float64
 8   Nonflavanoid     178 non-null    float64
 9   Proanthocyanins  178 non-null    float64
 10  Color            178 non-null    float64
 11  Hue              178 non-null    float64
 12  Dilution         178 non-null    float64
 13  Proline          178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [73]:
# View unique categories of target
wine['Type'].unique()

array([1, 2, 3])

In [74]:
# Checking how balanced is the data
wine['Type'].value_counts()

2    71
1    59
3    48
Name: Type, dtype: int64

Since data is not really balanced, we are going to use stratify later on.

1. One versus rest logistic regression model

In [75]:
# One vs Rest
y = wine['Type']
X = wine.drop(columns = 'Type')

In [76]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [77]:
# Make an instance of the model
logreg = LogisticRegression(C = 1)

In [78]:
# Scaling logistic regression
scaler = StandardScaler()

scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Training the model on the data
logreg.fit(X_train,y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
# Checking the scores
print('Training accuracy:', logreg.score(X_train, y_train))
print('Test accuracy:', logreg.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


This model gave us ideal results. 100% on training and 100% on testing. Let's try tuning this model.

In [80]:
logreg.get_params()

{'C': 1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [81]:
# Creating a dictionar with all params we'd like to tune
logreg_params = {'penalty' : ['l1', 'l2'],
                 'C' : [1, 25, 50, 100],
                 'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                 'multi_class' : ['ovr', 'multinomial']}

In [82]:
# Instantiate a gridsearch with the model and dict
grid = GridSearchCV(logreg, param_grid = logreg_params)
# Fitting the grid search model
grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 25, 50, 100],
                         'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_sc

In [83]:
# Check the optimal combinations of hyperparameters
grid.best_params_

{'C': 1, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'newton-cg'}

Looks like since our results for test and train are 100% there's no need of tuning this model.

2. One versus one logistic regression model

In [84]:
# One vs one. Making instance of a model. Scaler
ovo = OneVsOneClassifier(logreg)
scaler = StandardScaler()

In [85]:
#Pipeline
pipe = make_pipeline(scaler, ovo)

In [86]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('onevsoneclassifier',
                 OneVsOneClassifier(estimator=LogisticRegression(C=1,
                                                                 class_weight=None,
                                                                 dual=False,
                                                                 fit_intercept=True,
                                                                 intercept_scaling=1,
                                                                 l1_ratio=None,
                                                                 max_iter=100,
                                                                 multi_class='auto',
                                                                 n_jobs=None,
                                                                 penalty='l2',
                         

In [87]:
# Checking the scores
print('Training accuracy:', ovo.score(X_train, y_train))
print('Test accuracy:', ovo.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


Since this model gave us 100% accuracy with default I'm not going to tune it.

In [88]:
# L1 / using solver 'liblinear' since it needed for l1
logreg = LogisticRegression(penalty = 'l1', solver = 'liblinear')
logreg.fit(X_train, y_train)

print('Training accuracy:', logreg.score(X_train, y_train))
print('Test accuracy:', logreg.score(X_test, y_test))

Training accuracy: 0.9924812030075187
Test accuracy: 0.9777777777777777


In [89]:
# L2
logreg = LogisticRegression(penalty = 'l2')
logreg.fit(X_train, y_train)

print('Training accuracy:', logreg.score(X_train, y_train))
print('Test accuracy:', logreg.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


Comparing those two models we see using l2 giving us better result.

In [90]:
# KNN
knn = KNeighborsClassifier()

pipe_knn = make_pipeline(scaler, knn)

pipe_knn.fit(X_train, y_train)

print('Training accuracy:', pipe_knn.score(X_train, y_train))
print('Test accuracy:', pipe_knn.score(X_test, y_test))

Training accuracy: 0.9924812030075187
Test accuracy: 0.9333333333333333


In [91]:
# Bagged Tree
bt = BaggingClassifier()

pipe_bt = make_pipeline(scaler, bt)
pipe_bt.fit(X_train, y_train)

print('Training accuracy:', pipe_bt.score(X_train, y_train))
print('Test accuracy:', pipe_bt.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


In [92]:
# Random Forest
rf = RandomForestClassifier()

pipe_rf = make_pipeline(scaler, rf)
pipe_rf.fit(X_train, y_train)

print('Training accuracy:', pipe_rf.score(X_train, y_train))
print('Test accuracy:', pipe_rf.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


Out of those three models, Random Forest and Bagged Tree gave us 100& accuracy.

Which model performed the best? 

Logistic regression with l2 param and Random Forest


Which hyperparameters did you tune for each of the models? Is there a model that you liked the best and why?

I was looking for optimal params on logistic regression, but it seems like there's no need of that if it gives 100% accuracy with default params.

And I tried tunnung penalty and solver. I like Random Forest, because it seems to me to give the best results with bigger data sets, however it might take a lot of time. So if data set is really big, I would use knn.